In [ ]:
import jax.numpy as jnp
import jax.random as jrand
from jax import jit, vmap

from genjax import ChoiceMap, beta, flip, gen
from genjax.delayed import delay


@gen
def submodel():
    v = beta(2.0, 2.0) @ "v"
    return v


@gen
def model():
    v = submodel() @ "v"
    f = flip(v) @ "f"
    return f


# In future, logic goes into "GenerateRequest"...
(retval, chm), w = jit(vmap(delay(model.lower), in_axes=(0, None, None)))(
    jrand.split(jrand.key(1), 10), ChoiceMap.kw(f=jnp.array(True)), ()
)


def exact_beta_posterior_mean(c, a, b):
    return (a + c) / (a + b + c)


(chm, jnp.mean(chm["v", "v"]), exact_beta_posterior_mean(True, 2.0, 2.0))

In [ ]:
w